In [1]:
import numpy as np
import cv2
import math
#from queue import PriorityQueue
import time

In [2]:
env_map = np.zeros((100, 200))
env_map[:, :] = 0


matrix_dim_row = 200
matrix_dim_col = 200


In [20]:

class SimplePriorityQueue:
    def __init__(self):
        self.key_list = np.array([], dtype=object)
        self.value_list = np.array([], dtype=object)
    
    def put(self, key, value = None):
        self.key_list = np.append(self.key_list, key)
        self.value_list = np.append(self.value_list, value)
        
        ind = self.key_list.argsort(kind ='heapsort')
        self.key_list = self.key_list[ind]
        self.value_list = self.value_list[ind]
        
    def get(self):
        if len(self.key_list) != 0:
            key = self.key_list[0]
            value = self.value_list[0]
            self.key_list = np.delete(self.key_list, [0])
            self.value_list = np.delete(self.value_list, [0])
            return key, value
        else:
            return None, None
        
    def isempty(self):
        return len(self.key_list) == 0


In [17]:
#matrix_dim_row = 10
#matrix_dim_col = 20
def get_next_positions(node):
    ## add the cache logic
    
    ## for a 2D matrix (0,0) is there at the top left corner
    actions = {'U': [-1, 0], 'D': [1, 0], 'L': [0, -1], 'R': [0, 1], 
               'UL': [-1, -1], 'UR': [-1, 1], 'DL': [1, -1], 'DR': [1, 1]}
    
    ROOT_TWO = math.sqrt(2)
    action_cost_map = {'U': 1, 'D': 1, 'L': 1, 'R': 1, 
               'UL': ROOT_TWO, 'UR': ROOT_TWO, 'DL': ROOT_TWO, 'DR': ROOT_TWO}
    
    next_postions = []
    
    node_pos = list(node['pos'])
    for action in actions:
        next_pos = np.add(node_pos, actions[action])
        if is_position_valid(next_pos):
            node_path = node['path'].copy()
            node_path.append(action)
            
            next_postions.append({
                'pos': tuple(next_pos),
                'path': node_path,
                'cost': node['cost'] + action_cost_map[action]
            })
    return next_postions
            
    


def is_position_valid(position):
    ## rest of the constraints to be added - obstacle equations
    constraint_predicate1 = position[1] == 80 and (position[0] in range(50, 150))
    constraint_predicate2 = (position[1] in range(90, 120)) and (position[0] in range(40, 100))
    return (position[0] in range(0, matrix_dim_row) and position[1] in range(0, matrix_dim_col)) and (not constraint_predicate1) and (not constraint_predicate2)

In [15]:
def start_visualization(initial_pos, visited_nodes, path):
    try:
        env_map = np.zeros((matrix_dim_row, matrix_dim_col))
        env_map[:, :] = 0
        env_map[initial_pos] = 1
        cv2.imshow('Map', env_map)
        actions = {'U': [-1, 0], 'D': [1, 0], 'L': [0, -1], 'R': [0, 1], 
                   'UL': [-1, -1], 'UR': [-1, 1], 'DL': [1, -1], 'DR': [1, 1]}

        initial_pos = list(initial_pos)

        for node in visited_nodes:
            env_map[node] = 1
            cv2.imshow('Map', env_map)
            #time.sleep(0.001)

        next_pos = initial_pos
        for action in path:
            next_pos = np.add(next_pos, actions[action])
            env_map[tuple(next_pos)] = 0
            cv2.imshow('Map', env_map)
            #time.sleep(0.01)

        cv2.imshow('Map', env_map)
    except:
        print(-99999)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [21]:
cost_matrix = np.zeros((matrix_dim_row, matrix_dim_col), dtype=object)
cost_matrix[:, :] = math.inf

node_queue = SimplePriorityQueue()
visited_nodes = set()
initial_pos = (matrix_dim_row-1, 0)
target_pos = (0, matrix_dim_col-1)

initial_node = {'pos': initial_pos, 'path': [], 'cost': 0}
cost_matrix[initial_pos] = 0


is_target_found = False

# cost to reach to the node as key to sort
node_queue.put(0, initial_node)


#len(visited_nodes) < (matrix_dim_row*matrix_dim_col)

while (not node_queue.isempty()) and not is_target_found:
    current_node = node_queue.get()[1]
    if current_node['pos'] == target_pos:
        print('Found')
        is_target_found = True
        solution_path = current_node['path']
    else:
        visited_nodes.add(current_node['pos'])
        next_positions = get_next_positions(current_node)
        for node in next_positions:
            if node['pos'] not in visited_nodes:
                old_cost_to_reach_node = math.inf if cost_matrix[node['pos']] == math.inf else cost_matrix[node['pos']]['cost'] 
                new_cost_to_reach_node = node['cost']
                if old_cost_to_reach_node > new_cost_to_reach_node:
                    cost_matrix[node['pos']] = node
                    node_queue.put(node['cost'], node)

if is_target_found:
    print(solution_path)
    start_visualization(initial_pos, visited_nodes, solution_path)
else:
    print('not found')
                    

Found
['R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'U', 'U', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR', 'UR',

In [12]:
start_visualization(initial_pos, visited_nodes, solution_path)

In [24]:
a = np.array([9,3,2,6,7])

ind = a.argsort(kind ='heapsort')
ind

array([2, 1, 3, 4, 0], dtype=int64)

In [ ]:
if __name =='__main__':
    #R = c
    #Solver <-- C
    # solver.solve()